In [1]:
import cobra
from cobra.io import write_sbml_model
import pandas as pd
import sys
import numpy as np
from optlang.symbolics import add

In [64]:
model = cobra.io.read_sbml_model("../data/iDR741.xml")
model

Name,DR_draftmodel
Memory address,1e9818ee370
Number of metabolites,1106
Number of reactions,1148
Number of genes,740
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"c0, e0"


In [3]:
model.reactions

[<Reaction rxn05561_c0 at 0x1e9f5ecd100>,
 <Reaction rxn07917_c0 at 0x1e9f5ecdd60>,
 <Reaction rxn08553_c0 at 0x1e9f5ecdd90>,
 <Reaction rxn07918_c0 at 0x1e9f5ecdcd0>,
 <Reaction rxn07931_c0 at 0x1e9f5edfac0>,
 <Reaction rxn05205_c0 at 0x1e9f5edfc70>,
 <Reaction rxn10125_c0 at 0x1e9f5edfe20>,
 <Reaction rxn09211_c0 at 0x1e9f5edfee0>,
 <Reaction rxn02011_c0 at 0x1e9f5edf520>,
 <Reaction rxn01466_c0 at 0x1e9f5edf040>,
 <Reaction rxn04308_c0 at 0x1e9f5eecc10>,
 <Reaction rxn05293_c0 at 0x1e9f5eecfa0>,
 <Reaction rxn01213_c0 at 0x1e9f5eec5e0>,
 <Reaction rxn01220_c0 at 0x1e9f5eec8b0>,
 <Reaction rxn00077_c0 at 0x1e9f5eec940>,
 <Reaction rxn08018_c0 at 0x1e9f5ef5fa0>,
 <Reaction rxn05265_c0 at 0x1e9f5ef5580>,
 <Reaction rxn10336_c0 at 0x1e9f5ef5850>,
 <Reaction rxn09106_c0 at 0x1e9f5ef5be0>,
 <Reaction rxn09113_c0 at 0x1e9f5ef5460>,
 <Reaction rxn00247_c0 at 0x1e9f5efcf10>,
 <Reaction rxn02342_c0 at 0x1e9f5efcbb0>,
 <Reaction rxn01452_c0 at 0x1e9f5efc850>,
 <Reaction rxn00199_c0 at 0x1e9f5f

In [4]:
model.reactions.get_by_id("rxn05561_c0").gene_reaction_rule

'DR_RS12990'

In [5]:
model.reactions.get_by_id("rxn09449_c0").gene_reaction_rule

'DR_RS15135 and DR_RS01745 and DR_RS08640'

In [6]:
model.reactions.get_by_id("rxn10122_c0").gene_reaction_rule

'(DR_RS07710 and DR_RS07680 and DR_RS07650 and DR_RS07645 and DR_RS07665 and DR_RS07685 and DR_RS07675 and DR_RS07670 and DR_RS07640 and DR_RS07655 and DR_RS07660 and DR_RS07695 and DR_RS07705 and DR_RS07700) or DR_RS03765'

In [7]:
def create_gprdict(model):   
    gpr_dict = dict()
    for rxn in model.reactions:
        if rxn.gene_reaction_rule:
            temp = set()
            for x in [x.strip('() ') for x in rxn.gene_reaction_rule.split(' or ')]:
                temp.add(frozenset(y.strip('() ') for y in x.split(' and ')))
            gpr_dict[rxn.id] = temp
    return gpr_dict

In [71]:
def transcript_value_for_rxn(model, transcriptomics_df, rxn):
    final_transcript_value = 0
    #gene_ids = []
    for parallel_gene in create_gprdict(model)[rxn.id]:
        transcript_values = []
        for gene in parallel_gene:
            if gene in transcriptomics_df.index:
                transcript_values.append(transcriptomics_df.loc[gene].to_numpy()[0])
                #print(transcriptomics_df.loc[gene].to_numpy()[0])
            else:
                transcript_values.append(np.inf)
            min_transcript_val = np.min(transcript_values)
        final_transcript_value = final_transcript_value + min_transcript_val
    # 取整
    final_transcript_value = round(final_transcript_value)
    
    return final_transcript_value

In [9]:
def EFlux2(model, Transcriptomics):
    # copy model and set tolerance
    eflux2_model = model.copy()
    eflux2_model.tolerance = 1e-9
    
    # set the flux bounds for each reaction using the transcriptomics data    
    for rxn in eflux2_model.reactions:
        if 'EX_' not in str(rxn.id):
            if rxn.gene_reaction_rule:
                if rxn.lower_bound < 0.0:
                    rxn.lower_bound = -transcript_value_for_rxn(model, Transcriptomics, rxn)
                else:
                    pass
                if rxn.upper_bound > 0.0:
                    rxn.upper_bound = transcript_value_for_rxn(model, Transcriptomics, rxn)
                else:
                    pass
            else:
                """When there is no GPR, the arbitrary bounds are removed. 
                Common arbitrary bound value of 1000 for E.coli, might be different depending on the model, e.g., 99999.0 for iMM904 yeast model in BiGG"""
                if rxn.lower_bound <= -1000:
                    rxn.lower_bound = -np.Inf
                if rxn.upper_bound >= 1000:
                    rxn.upper_bound = np.Inf 
    
    # solve FBA problem with transcriptomic bounds
    fba_solution = eflux2_model.optimize()
    print('FBA status', fba_solution.status)
    print('FBA solution', fba_solution.objective_value)
    
    #display(eflux2_model.summary(solution=fba_solution))
        
    return fba_solution

In [79]:
def EFlux2_model(model, Transcriptomics):
    # copy model
    eflux2_model = model.copy()
    
    # set the flux bounds for each reaction using the transcriptomics data    
    for rxn in eflux2_model.reactions:
        if 'EX_' not in str(rxn.id):
            if rxn.gene_reaction_rule:
                if rxn.lower_bound < 0.0:
                    rxn.lower_bound = -transcript_value_for_rxn(model, Transcriptomics, rxn)
                else:
                    pass
                if rxn.upper_bound > 0.0:
                    rxn.upper_bound = transcript_value_for_rxn(model, Transcriptomics, rxn)
                else:
                    pass
        
    return eflux2_model

In [10]:
transcript_df_wt = pd.read_csv('../data/gene_fpkm.csv', index_col='extracted_gene_id', 
                               usecols=['extracted_gene_id', 'wt_fpkm_average'])
transcript_df_mutant = pd.read_csv('../data/gene_fpkm.csv', index_col='extracted_gene_id', 
                            usecols=['extracted_gene_id', 'mutant_fpkm_average'])
transcript_df_wt

,wt_fpkm_average
extracted_gene_id,
DR_RS14970,1867.117716
DR_RS02940,38.272997
DR_RS10920,559.391256
DR_RS06790,986.889364
DR_RS10060,126.177091
...,...
DR_RS03665,20.462311
DR_RS13540,86.830564
DR_RS14985,45.778937


In [14]:
flux_solution_wt = EFlux2(model, transcript_df_wt)
flux_solution_mutant = EFlux2(model, transcript_df_mutant)

FBA status optimal
FBA solution 4.69519957490637
FBA status optimal
FBA solution 4.490290459379256


In [19]:
def cobra_solution_to_df(model, solution):
    fluxes = []
    for rxn_id, flux in solution.fluxes.items():
        fluxes.append({
            'reaction_id': rxn_id,
            'reaction_name': model.reactions.get_by_id(rxn_id).name,
            'reaction_reaction': model.reactions.get_by_id(rxn_id).reaction,
            'flux': flux
        })
    return pd.DataFrame(fluxes)

In [21]:
flux_df_wt = cobra_solution_to_df(model, flux_solution_wt)
flux_df_wt.to_excel('genome_scale_fluxes/wt.xlsx', index=False)

In [22]:
flux_df_mutant = cobra_solution_to_df(model, flux_solution_mutant)
flux_df_mutant.to_excel('genome_scale_fluxes/mutant.xlsx', index=False)

In [80]:
flux_model_wt = EFlux2_model(model, transcript_df_wt)
flux_model_mutant = EFlux2_model(model, transcript_df_mutant)

write_sbml_model(flux_model_wt, "./genome_scale_fluxes/wt_fbc.xml")
write_sbml_model(flux_model_mutant, "./genome_scale_fluxes/mutant_fbc.xml")